In [28]:
import graphviz
import warnings

# Suppress any potential deprecation warnings from the library itself for a cleaner output
warnings.filterwarnings("ignore", category=DeprecationWarning)

def create_unet_diagram():
    """Generates a compact Graphviz diagram for the custom UNet architecture without skip labels."""
    dot = graphviz.Digraph('UNet_Architecture', comment='3D UNet')
    dot.attr(rankdir='TB', splines='ortho', nodesep='0.15', ranksep='0.3')
    dot.attr('node', shape='box', style='rounded,filled', fillcolor='lightblue', fontname='Helvetica')
    dot.attr('edge', fontname='Helvetica', fontsize='9')
    dot.attr(label='3D UNet Architecture', fontsize='16', fontname='Helvetica-Bold')
    dot.attr(dpi='1000')

    # Node styles
    conv_style = {'fillcolor': '#a3d9ff'}
    pool_style = {'fillcolor': '#ffaaaa', 'shape': 'box'}
    upconv_style = {'fillcolor': '#a3ffb3'}
    output_style = {'fillcolor': '#d3a3ff'}

    # Input / Output
    dot.node('input', 'INPUT\n(4, 128, 128, 128)', shape='box3d', fillcolor='gray88')
    dot.node('output', 'OUTPUT\n(5, 128, 128, 128)', shape='box3d', fillcolor='gray88')

    # Encoder Path
    with dot.subgraph(name='cluster_encoder') as c:
        c.attr(label='Encoder (Contracting Path)', style='rounded', color='gray')
        c.node('enc0', 'Stage 0\nConv(4→32) x2\nInstanceNorm, LeakyReLU', **conv_style)
        c.node('pool0', 'MaxPool (2x2x2)', **pool_style)
        c.node('enc1', 'Stage 1\nConv(32→64) x2\nInstanceNorm, LeakyReLU', **conv_style)
        c.node('pool1', 'MaxPool (2x2x2)', **pool_style)
        c.node('enc2', 'Stage 2\nConv(64→128) x2\nInstanceNorm, LeakyReLU', **conv_style)
        c.node('pool2', 'MaxPool (2x2x2)', **pool_style)
        c.node('enc3', 'Stage 3\nConv(128→256) x2\nInstanceNorm, LeakyReLU', **conv_style)
        c.node('pool3', 'MaxPool (2x2x2)', **pool_style)
        c.node('bottleneck', 'Bottleneck\nConv(256→320) x2\nInstanceNorm, LeakyReLU', **conv_style)

    dot.edge('input', 'enc0', label='(128³)')
    dot.edge('enc0', 'pool0')
    dot.edge('pool0', 'enc1', label='(64³)')
    dot.edge('enc1', 'pool1')
    dot.edge('pool1', 'enc2', label='(32³)')
    dot.edge('enc2', 'pool2')
    dot.edge('pool2', 'enc3', label='(16³)')
    dot.edge('enc3', 'pool3')
    dot.edge('pool3', 'bottleneck', label='(8³)')

    # Decoder Path
    with dot.subgraph(name='cluster_decoder') as c:
        c.attr(label='Decoder (Expanding Path)', style='rounded', color='gray')
        c.node('up3', 'UpConv (320→256)', **upconv_style)
        c.node('dec3', 'Stage 3\nConcat(256+256)→512\nConv(512→256) x2', **conv_style)
        c.node('up2', 'UpConv (256→128)', **upconv_style)
        c.node('dec2', 'Stage 2\nConcat(128+128)→256\nConv(256→128) x2', **conv_style)
        c.node('up1', 'UpConv (128→64)', **upconv_style)
        c.node('dec1', 'Stage 1\nConcat(64+64)→128\nConv(128→64) x2', **conv_style)
        c.node('up0', 'UpConv (64→32)', **upconv_style)
        c.node('dec0', 'Stage 0\nConcat(32+32)→64\nConv(64→32) x2', **conv_style)

    dot.edge('bottleneck', 'up3', label='(8³)')
    dot.edge('up3', 'dec3', label='(16³)')
    dot.edge('dec3', 'up2')
    dot.edge('up2', 'dec2', label='(32³)')
    dot.edge('dec2', 'up1')
    dot.edge('up1', 'dec1', label='(64³)')
    dot.edge('dec1', 'up0')
    dot.edge('up0', 'dec0', label='(128³)')

    # --- Skip Connections (Labels Removed) ---
    dot.edge('enc3', 'dec3', style='dashed', constraint='false')
    dot.edge('enc2', 'dec2', style='dashed', constraint='false')
    dot.edge('enc1', 'dec1', style='dashed', constraint='false')
    dot.edge('enc0', 'dec0', style='dashed', constraint='false')

    # Final Output
    dot.node('out_conv', 'Output Conv (32→5)\nKernel (1x1x1)', **output_style)
    dot.edge('dec0', 'out_conv')
    dot.edge('out_conv', 'output')
    
    return dot

def create_segresnet_diagram():
    """Generates a compact Graphviz diagram for the MONAI SegResNet architecture without skip labels."""
    dot = graphviz.Digraph('SegResNet_Architecture', comment='MONAI SegResNet')
    dot.attr(rankdir='TB', splines='ortho', nodesep='0.15', ranksep='0.3')
    dot.attr('node', shape='box', style='rounded,filled', fillcolor='lightgreen', fontname='Helvetica')
    dot.attr('edge', fontname='Helvetica', fontsize='9')
    dot.attr(label='MONAI SegResNet Architecture', fontsize='16', fontname='Helvetica-Bold')
    dot.attr(dpi='1000')

    # Node styles
    conv_style = {'fillcolor': '#c1ffc1'}
    down_style = {'fillcolor': '#ffc1c1', 'shape': 'box'}
    up_style = {'fillcolor': '#c1c1ff'}
    add_style = {'fillcolor': '#f9f970', 'shape': 'circle', 'label': '+'}
    output_style = {'fillcolor': '#e6c1ff'}

    # Input / Output
    dot.node('input', 'INPUT\n(4, 128, 128, 128)', shape='box3d', fillcolor='gray88')
    dot.node('output', 'OUTPUT\n(3, 128, 128, 128)', shape='box3d', fillcolor='gray88')

    # Initial Convolution
    dot.node('conv_init', 'Initial Conv (4→16)\nGroupNorm, ReLU', **conv_style)
    dot.edge('input', 'conv_init')

    # Encoder Path
    with dot.subgraph(name='cluster_encoder') as c:
        c.attr(label='Encoder (Down Blocks)', style='rounded', color='gray')
        c.node('enc0', 'Stage 0\n1x ResBlock (16ch)', **conv_style)
        c.node('down1', 'Strided Conv (16→32)', **down_style)
        c.node('enc1', 'Stage 1\n2x ResBlock (32ch)', **conv_style)
        c.node('down2', 'Strided Conv (32→64)', **down_style)
        c.node('enc2', 'Stage 2\n2x ResBlock (64ch)', **conv_style)
        c.node('down3', 'Strided Conv (64→128)', **down_style)
        c.node('enc3', 'Stage 3\n4x ResBlock (128ch)', **conv_style)

    dot.edge('conv_init', 'enc0', label='(128³)')
    dot.edge('enc0', 'down1')
    dot.edge('down1', 'enc1', label='(64³)')
    dot.edge('enc1', 'down2')
    dot.edge('down2', 'enc2', label='(32³)')
    dot.edge('enc2', 'down3')
    dot.edge('down3', 'enc3', label='(16³)')

    # Decoder Path
    with dot.subgraph(name='cluster_decoder') as c:
        c.attr(label='Decoder (Up Blocks)', style='rounded', color='gray')
        c.node('up2', 'Upsample (128→64)\n1x1 Conv + Trilinear', **up_style)
        c.node('add2', **add_style)
        c.node('dec2', 'Stage 2\n1x ResBlock (64ch)', **conv_style)
        c.node('up1', 'Upsample (64→32)\n1x1 Conv + Trilinear', **up_style)
        c.node('add1', **add_style)
        c.node('dec1', 'Stage 1\n1x ResBlock (32ch)', **conv_style)
        c.node('up0', 'Upsample (32→16)\n1x1 Conv + Trilinear', **up_style)
        c.node('add0', **add_style)
        c.node('dec0', 'Stage 0\n1x ResBlock (16ch)', **conv_style)

    dot.edge('enc3', 'up2', label='(16³)')
    dot.edge('up2', 'add2', label='(32³)')
    dot.edge('add2', 'dec2')
    dot.edge('dec2', 'up1')
    dot.edge('up1', 'add1', label='(64³)')
    dot.edge('add1', 'dec1')
    dot.edge('dec1', 'up0')
    dot.edge('up0', 'add0', label='(128³)')
    dot.edge('add0', 'dec0')

    # --- Skip Connections (Labels Removed) ---
    dot.edge('enc2', 'add2', style='dashed', constraint='false')
    dot.edge('enc1', 'add1', style='dashed', constraint='false')
    dot.edge('enc0', 'add0', style='dashed', constraint='false')

    # Final Output
    dot.node('conv_final', 'Final Block\nGroupNorm, ReLU\n1x1 Conv (16→3)', **output_style)
    dot.edge('dec0', 'conv_final')
    dot.edge('conv_final', 'output')

    return dot

# Generate and save the diagrams
unet_graph = create_unet_diagram()
unet_graph.render('unet_architecture_final', format='png', view=False, cleanup=True)

segresnet_graph = create_segresnet_diagram()
segresnet_graph.render('segresnet_architecture_final', format='png', view=False, cleanup=True)

print("Final diagrams 'unet_architecture_final.png' and 'segresnet_architecture_final.png' have been generated.")

Final diagrams 'unet_architecture_final.png' and 'segresnet_architecture_final.png' have been generated.


In [30]:
import graphviz

def create_modified_model_diagram():
    """Generates the Graphviz diagram for the modified MONAI pre-trained model."""
    dot = graphviz.Digraph('Modified_MONAI_Architecture', comment='Transfer Learning Model')
    dot.attr(rankdir='TB', splines='ortho', nodesep='0.15', ranksep='0.3')
    dot.attr('node', shape='box', style='rounded,filled', fontname='Helvetica')
    dot.attr('edge', fontname='Helvetica', fontsize='8')
    dot.attr(label='Modified Pre-trained MONAI Architecture', fontsize='16', fontname='Helvetica-Bold')
    dot.attr(dpi='1000')

    # --- Node Styles for Different States ---
    frozen_style = {'fillcolor': '#d1e0e0', 'fontcolor': '#555555'} # Muted gray-cyan for frozen layers
    trainable_style = {'fillcolor': '#c1ffc1'} # Green for trainable layers
    modified_style = {'fillcolor': '#e6c1ff', 'style': 'rounded,filled,bold', 'color': 'purple'} # Purple for new/replaced layers
    io_style = {'shape': 'box3d', 'fillcolor': 'gray88'}

    # Input / Output
    dot.node('input', 'INPUT\n(4, 224, 224, 144)', **io_style)
    dot.node('output', 'OUTPUT\n(5, 224, 224, 144)', **io_style)

    # Encoder Path (Frozen)
    with dot.subgraph(name='cluster_encoder') as c:
        c.attr(label='Encoder (Pre-trained & Frozen)', style='rounded', color='gray')
        c.node('enc1', 'Block 1\nResidualBlock (4→32)\n+ Downsample (2x)', **frozen_style)
        c.node('enc2', 'Block 2\nResidualBlock (32→64)\n+ Downsample (2x)', **frozen_style)
        c.node('enc3', 'Block 3\nResidualBlock (64→128)\n+ Downsample (2x)', **frozen_style)
        c.node('enc4', 'Block 4\nResidualBlock (128→256)\n+ Downsample (2x)', **frozen_style)
        c.node('bottleneck', 'Bottleneck\nResidualBlock (256→512)', **frozen_style)

    dot.edge('input', 'enc1', label='(224, 224, 144)')
    dot.edge('enc1', 'enc2', label='(112, 112, 72)')
    dot.edge('enc2', 'enc3', label='(56, 56, 36)')
    dot.edge('enc3', 'enc4', label='(28, 28, 18)')
    dot.edge('enc4', 'bottleneck', label='(14, 14, 9)')

    # Decoder Path (Trainable)
    with dot.subgraph(name='cluster_decoder') as c:
        c.attr(label='Decoder (Trainable)', style='rounded', color='gray')
        c.node('dec4', 'Block 4\nUpsample (2x)\n+ ResidualBlock (512→256)', **trainable_style)
        c.node('dec3', 'Block 3\nUpsample (2x)\n+ ResidualBlock (256→128)', **trainable_style)
        c.node('dec2', 'Block 2\nUpsample (2x)\n+ ResidualBlock (128→64)', **trainable_style)
        c.node('dec1', 'Block 1\nUpsample (2x)\n+ ResidualBlock (64→32)', **trainable_style)

    dot.edge('bottleneck', 'dec4', label='(14, 14, 9)')
    dot.edge('dec4', 'dec3', label='(28, 28, 18)')
    dot.edge('dec3', 'dec2', label='(56, 56, 36)')
    dot.edge('dec2', 'dec1', label='(112, 112, 72)')

    # Skip Connections
    dot.edge('enc4', 'dec4', style='dashed', constraint='false')
    dot.edge('enc3', 'dec3', style='dashed', constraint='false')
    dot.edge('enc2', 'dec2', style='dashed', constraint='false')
    dot.edge('enc1', 'dec1', style='dashed', constraint='false')

    # Modified Output Head
    dot.node('modified_head', '🔧 MODIFIED OUTPUT HEAD\n(Replaced & Trainable)\n\nOriginal: Conv3D(32→3)\nNew: Conv3D(32→5)', **modified_style)
    dot.edge('dec1', 'modified_head', label='(224, 224, 144)')
    dot.edge('modified_head', 'output')
    
    return dot

def create_residual_block_diagram():
    """Generates a detailed diagram of the Residual Block pattern."""
    dot = graphviz.Digraph('Residual_Block_Pattern', comment='Residual Block Detail')
    dot.attr(rankdir='LR', splines='curved') # Horizontal layout
    dot.attr('node', shape='box', style='rounded,filled', fontname='Helvetica')
    dot.attr(label='Residual Block Pattern', fontsize='16', fontname='Helvetica-Bold')
    dot.attr(dpi='1000')

    # Node styles
    op_style = {'fillcolor': '#a3d9ff'}
    norm_style = {'fillcolor': '#c1ffc1'}
    relu_style = {'fillcolor': '#f9f970', 'shape': 'diamond'}
    add_style = {'fillcolor': '#ffaaaa', 'shape': 'circle', 'label': '+'}
    io_style = {'shape': 'ellipse', 'style': 'filled', 'fillcolor': 'gray88'}

    # Nodes
    dot.node('input', 'Input', **io_style)
    dot.node('conv1', 'Conv3D', **op_style)
    dot.node('norm1', 'GroupNorm', **norm_style)
    dot.node('relu1', 'ReLU', **relu_style)
    dot.node('conv2', 'Conv3D', **op_style)
    dot.node('norm2', 'GroupNorm', **norm_style)
    dot.node('add', **add_style)
    dot.node('relu_out', 'ReLU', **relu_style)
    dot.node('output', 'Output', **io_style)

    # Main Path
    dot.edge('input', 'conv1')
    dot.edge('conv1', 'norm1')
    dot.edge('norm1', 'relu1')
    dot.edge('relu1', 'conv2')
    dot.edge('conv2', 'norm2')
    dot.edge('norm2', 'add')
    dot.edge('add', 'relu_out')
    dot.edge('relu_out', 'output')
    
    # Residual (Skip) Path
    dot.edge('input', 'add', constraint='false', minlen='2')
    
    return dot

# --- Generate and Save Diagrams ---
modified_model_graph = create_modified_model_diagram()
modified_model_graph.render('modified_monai_architecture', format='png', view=False, cleanup=True)

residual_block_graph = create_residual_block_diagram()
residual_block_graph.render('residual_block_pattern', format='png', view=False, cleanup=True)

print("Diagrams 'modified_monai_architecture.png' and 'residual_block_pattern.png' have been generated.")

Diagrams 'modified_monai_architecture.png' and 'residual_block_pattern.png' have been generated.
